# Plotly Interactive Plots

Some interactive plots to visualize the dataset with more details.

In [1]:
# imports

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# plotly imports
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Test API function before running larger simulations
from examples.sim_trace import generate_trace_api

input_data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 10000,
}

generate_trace_api(input_data)

{'reqs_cold': 15,
 'reqs_total': 8955,
 'reqs_warm': 8940,
 'prob_cold': 0.0016750418760469012,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': 5046.113497120165,
 'inst_count_avg': 7.495944792713149,
 'inst_running_count_avg': 1.782058906803205,
 'inst_idle_count_avg': 5.713885885909943,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 10000}

In [3]:
from pacssim.ServerlessSimulator import ServerlessSimulator

sim = ServerlessSimulator(**input_data)
sim.generate_trace(debug_print=False, progress=False)

hist_idx = 0
# go in 10 second steps
hist_step = sim.max_time / 200
sampled_hist_times = [sim.hist_times[0]]
sampled_hist_inst_counts = [sim.hist_server_count[0]]
while hist_idx < (len(sim.hist_times) - 1):
    hist_idx += 1
    
    if sim.hist_times[hist_idx] - sampled_hist_times[-1] < hist_step:
        continue
    
    sampled_hist_times.append(sim.hist_times[hist_idx])
    sampled_hist_inst_counts.append(sim.hist_server_count[hist_idx])

# calculate sampled instance count average so far
sampled_hist_inst_avgs = np.cumsum(sampled_hist_inst_counts) / np.array(list(range(len(sampled_hist_inst_counts))))

In [4]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = [i/60 for i in sampled_hist_times],
        y = sampled_hist_inst_counts,
        mode = 'markers+lines',
        name = "Current Value",
    ),
)
fig.add_trace(
    go.Scatter(
        x = [i/60 for i in sampled_hist_times],
        y = sampled_hist_inst_avgs,
        mode = 'markers+lines',
        name = "Average Estimate",
    ),
)
fig.update_layout(title="Instance Counts Over Time", xaxis=dict(title="Time (minutes)"), yaxis=dict(title="Instance Count"))

NameError: name 'go' is not defined

## Make The Interactive Overall Plots

In [5]:
# Plot characteristics for different arrival rates and expiration thresholds
num_arrival_rates = 10
exp_thresholds = [60, 600, 1200, 1800]
exp_threshold_labels = ["1 min", "10 min", "20 min", "30 min"]
input_data['arrival_rate'] = list(np.repeat(np.logspace(-3,1,num_arrival_rates), len(exp_thresholds)))
input_data['expiration_threshold'] = exp_thresholds * num_arrival_rates
input_data['expiration_threshold_labels'] = exp_threshold_labels * num_arrival_rates

df = pd.DataFrame(data=input_data).reset_index(drop=True)

def generate_trace_apply(x):
    return pd.Series(generate_trace_api(x.to_dict()))

df = df.progress_apply(generate_trace_apply, axis=1)
df.tail()

95%|█████████▌| 38/40 [00:21<00:01,  1.79it/s]


KeyboardInterrupt: 

In [4]:
# calculate utilizations
df['utilization_percent'] = df['inst_running_count_avg'] / df['inst_count_avg'] * 100
df['prob_cold_percent'] = df['prob_cold'] * 100

## Interactive Plots

We will be making interactive plots here using the data generated by our simulator.

In [5]:
# simple interactive plot

# create a dataframe for each expiration threshold
dfs = [df.loc[df['expiration_threshold_labels'] == name, :] for name in exp_threshold_labels]

# create traces
def df_add_trace(col_name, col_label, fig, axis_titles):
    for idx in range(len(exp_threshold_labels)):
        fig.add_trace(
            go.Scatter(
                x = dfs[idx].loc[:,'arrival_rate'],
                y = dfs[idx].loc[:,col_name],
                mode = 'markers+lines',
                name = exp_threshold_labels[idx],
            ),
        )

        axis_postfix = ""
        axis_titles["xaxis" + axis_postfix] = dict(title="Arrival Rate (reqs/sec)")
        axis_titles["yaxis" + axis_postfix] = dict(title=col_label)

    return axis_titles


def plot_fig_for_col(col_name, col_label):
    fig = go.Figure()
    axis_titles = {}
    axis_titles = df_add_trace(col_name, col_label, fig, axis_titles)
    fig.update_layout(
        xaxis_type="log",
        **axis_titles
        )
    fig.show()

In [6]:
plot_fig_for_col('prob_cold_percent', 'Prob of Cold Start (%)')

In [7]:
plot_fig_for_col('utilization_percent', 'Utilization (%)')

In [8]:
plot_fig_for_col('inst_idle_count_avg', 'Average Idle Instances')